In [ ]:
# load image
from func.utils import get_model_output_id_wnid_class_dict # get mapping: format: {"Model Ouput ID": ["WNID", "Class"]}
from func.utils import get_imagenet_id_wnid_class_dict # get mapping: format: {"ImageNet ID": ["WNID", "class"]}, e.g. {..."233": ['n02106382', 'Bouvier_des_Flandres'], ...}
from func.utils import map_model_id_to_imagenet_id, map_imagenet_id_to_model_id # mapping funcs

from func.utils import save_obj, load_obj

from func.saliency_maps import conduct_saliency_map_method, GuidedBackprop, VanillaBackprop, SmoothGrad, GradCAM, GuidedGradCAM, IntegratedGradients, GradientxInput

from func.responsible_regions import load_responsible_regions_from_given_path, X_y_preparation, process_cat_saliency_map

from func.concept_classifier import get_linear_classifier, get_xgb_classifier, prediction

from func.show import load_feature_saliency_map_and_resized_img_for_show, show_concept_region_on_img

# from func.receptive_field import receptive_field, get_rfs_from_a_mask, show_rf_in_org_img, show_some_rfs_randomly, show_some_rfs_order, get_rfs_from_a_mask_with_order

import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import gc

import torch
from torchvision import models

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# get the dict of ImageNet ID, WNID and class name
# format: {"ImageNet ID": ["WNID", "class"]}, e.g. {..."233": ['n02106382', 'Bouvier_des_Flandres'], ...}
imagenet_id_label=get_imagenet_id_wnid_class_dict(matfilepath = "imagenet_info/ILSVRC2012_meta.mat")

# get the dict of model output ID, WNID and class name
# format: {"Model Ouput ID": ["WNID", "Class"]}
modeloutput_id_label=get_model_output_id_wnid_class_dict(jsonfilepath = "imagenet_info/imagenet_label_index.json")

# get dict map model output ID to ImageNet ID
map_dict_model2imagenet=map_model_id_to_imagenet_id(imagenet_id_label, modeloutput_id_label)

# get ImageNet ID to dict map model output ID
map_dict_imagenet2model=map_imagenet_id_to_model_id(imagenet_id_label, modeloutput_id_label)

### show imagenet classes

In [ ]:
# only 1~1000 is valid
for idx in imagenet_id_label:
    print(str(idx)+": "+str(imagenet_id_label[idx]))

### imagenet parent and child

In [ ]:
imagenet_class_parent_and_child_dict = load_obj("imagenet_info/imagenet_class_dict")

In [ ]:
# e.g.
imagenet_class_parent_and_child_dict[1095]

### Responisble regions

In [ ]:
# set the path to load feature maps and saliency maps of a model
feature_maps_and_saliency_maps_path = "output/feature_maps_and_saliency_maps_vgg19"

In [ ]:
# set ID_groups and layers
ID_groups = [[1082,1095,1845]]
layers = ["features.30"]

In [ ]:
# set the path to save responisble regions
responisble_regions_save_path = "output/responisble_regions"

if not os.path.exists(responisble_regions_save_path):
    os.mkdir(responisble_regions_save_path)

In [ ]:
IDs_str = []
for ID_group in ID_groups:
    IDs_str.append(str(ID_group))
IDs_str = np.array(IDs_str)

for ID_group in ID_groups:
    for ID in ID_group:
        print(ID, imagenet_class_parent_and_child_dict[ID]['words'], imagenet_class_parent_and_child_dict[ID]['gloss'])

In [ ]:
for idx_layer, layer in enumerate(layers):
    chosen_layer = layer
    
    save_layer_path = os.path.join(responisble_regions_save_path, chosen_layer)
    if not os.path.exists(save_layer_path):
        os.mkdir(save_layer_path)
    
    X_dict = {}
    for ID_group in ID_groups:
        for idx, ID in enumerate(ID_group):
            print("loading data", chosen_layer, ID, imagenet_class_parent_and_child_dict[ID]['words'], imagenet_class_parent_and_child_dict[ID]['gloss'])
            if ID not in X_dict.keys():
                X_pos_current, X_neg_current = load_responsible_regions_from_given_path(os.path.join(feature_maps_and_saliency_maps_path, "result_of_ID_"+str(ID)),
                                                                                        layer = chosen_layer, pic_size = 14)
                X_dict[ID] = {}
                X_dict[ID]["foreground"] = X_pos_current
                X_dict[ID]["background"] = X_neg_current
    
    for idx_ID, ID_group in enumerate(ID_groups):
        which_ID_group = idx_ID
        save_layer_ID_group_path = os.path.join(save_layer_path, IDs_str[which_ID_group])
        if not os.path.exists(save_layer_ID_group_path):
            os.mkdir(save_layer_ID_group_path)

        flag = True
        idx = 0

        print("\nFor "+save_layer_ID_group_path+"/X_y.npz, the labels: ",end="\n\n")
        for position_in_group, ID in enumerate(ID_group):
            X_pos_current = X_dict[ID]['foreground']
            X_neg_current = X_dict[ID]['background']
            
            print(str(idx)+" presents concept "+str(ID)+" which is "+str(imagenet_class_parent_and_child_dict[ID]), end="\n\n")
            if flag:
                X = X_pos_current
                y = np.ones(X_pos_current.shape[0]) * idx
                X_neg = X_neg_current
                flag = False
                idx+=1
            else:
                X = np.concatenate((X, X_pos_current))
                y = np.concatenate((y, np.ones(X_pos_current.shape[0]) * idx))
                X_neg = np.concatenate((X_neg, X_neg_current))
                idx+=1
        
        print(str(idx)+" presents background info")
        X_neg = X_neg[np.random.randint(X_neg.shape[0], size=int(X.shape[0]/len(ID_group))),:]
        X = np.concatenate((X, X_neg))
        y = np.concatenate((y, np.ones(X_neg.shape[0]) * (idx)))
        
        np.savez_compressed(save_layer_ID_group_path+"/X_y.npz", X=X, y=y)

### Concept classifier and Shap values

In [ ]:
chosen_layer = str(layers[0])
print("chosen layer: "+chosen_layer)

chosen_ID_group = str(ID_groups[0])
print("chosen_ID_group: "+chosen_ID_group)

X_y = np.load(os.path.join(responisble_regions_save_path, chosen_layer, chosen_ID_group, "X_y.npz"))

In [ ]:
target_obj_loc = 1

In [ ]:
y = copy.deepcopy(X_y["y"])
print("target is "+str(chosen_ID_group.split(",")[target_obj_loc])+" | "+imagenet_class_parent_and_child_dict[int(chosen_ID_group.split(",")[target_obj_loc])]['words'])
print("we will train a concept classifier to distinguish the target from the others in "+str(chosen_ID_group))
y[y!=target_obj_loc] = -1
y[y==target_obj_loc] = 1
y[y!=1] = 0

In [ ]:
# train the concept classifier
coef, clf, Validation_score = get_linear_classifier(X_y["X"], y, classifier = "SGD")

In [ ]:
bst = get_xgb_classifier(X_y["X"],y)
bst.set_param({"predictor": "gpu_predictor"})
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(X_y["X"])

In [ ]:
importance_arr_mean = []
importance_arr_median = []
importance_arr_max = []
importance_arr_min = []

for i in range(shap_values.shape[1]):
    shap_vals_of_one_channel = np.abs(shap_values[:,i]) #xgb_shap_values[:,i]
    importance_arr_mean.append(np.mean(shap_vals_of_one_channel))
    importance_arr_median.append(np.median(shap_vals_of_one_channel))
    importance_arr_max.append(np.max(shap_vals_of_one_channel))
    importance_arr_min.append(np.min(shap_vals_of_one_channel))

importance_arr_mean = np.array(importance_arr_mean)
importance_arr_median = np.array(importance_arr_median)
importance_arr_max = np.array(importance_arr_max)
importance_arr_min = np.array(importance_arr_min)

In [ ]:
# plot shap vals

sort_locs = np.argsort(importance_arr_mean)[::-1]

sort_locs_str = []
for i in sort_locs:
     sort_locs_str.append(str(i))

N_subplot = 7
len_of_one_sub = int(len(sort_locs)/N_subplot+1)
sort_locs_sub = []
sort_locs_str_sub = []
for i in range(N_subplot):
    start_loc = np.clip(i*len_of_one_sub,0,len(sort_locs))
    end_loc = np.clip((i+1)*len_of_one_sub,0,len(sort_locs))
    sort_locs_sub.append(sort_locs[start_loc:end_loc])
    sort_locs_str_sub.append(sort_locs_str[start_loc:end_loc])
    
fig, axs = plt.subplots(1,N_subplot,figsize=(20,20))

for i in range(N_subplot):
    axs[i].barh(sort_locs_str_sub[i][::-1],importance_arr_mean[sort_locs_sub[i]][::-1])
    axs[i].set_xlim(0, np.max(importance_arr_mean)+0.05)
    axs[i].set_xlabel('Shapley')
# plt.savefig("shap_bar_plot.svg",
#             bbox_inches='tight', dpi=100, pad_inches=0.0)

### Show the results of concept classifier

In [ ]:
ID_for_show = 1095

print(ID_for_show, imagenet_class_parent_and_child_dict[ID_for_show]['words'])

resized_img_for_show_cat, feature_map_for_show_cat, saliency_map_for_show_cat = \
load_feature_saliency_map_and_resized_img_for_show(feature_maps_and_saliency_maps_path+"/result_of_ID_"+str(ID_for_show), chosen_layer)

resized_img_for_show_cat = np.array(resized_img_for_show_cat, dtype=int)

grayscale_saliency_map = process_cat_saliency_map(saliency_map_for_show_cat, num_of_pic_of_a_row = 10, mode="norm")

concept_map = prediction(clf, feature_map_for_show_cat, is_predict_proba = True)

In [ ]:
show_concept_region_on_img(resized_img_for_show_cat, concept_map[:,:,1])